In [66]:
import os
import math
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from datasets import load_dataset
from transformers import AutoTokenizer
# reproducibility
torch.manual_seed(123)

In [67]:
# Parameters (script defaults)
DATASET = "imdb"
TOKENIZER = "bert-base-uncased"
MAX_LEN = 64
HIDDEN_DIM = 128
NUM_LAYERS = 3
TRAIN_SAMPLES = 2000
TEST_SAMPLES = 256
BATCH_SIZE = 64
EPOCHS = 50
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Validation / early stopping
VAL_RATIO = 0.1
PATIENCE = 5
# Dropout (applied between layers during training only)
DROPOUT_RATE = 0.3

# Reproduce default MLP from `train_export.py`
This notebook splits parameter definition, imports, dataset prep, tokenization, training, and testing into separate cells.

In [68]:
class LinearMLP(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int, output_dim: int, num_layers: int, dropout: float = 0.0):
        super().__init__()
        if num_layers < 1:
            raise ValueError("num_layers must be >= 1")
        layers = []
        if num_layers == 1:
            layers.append(nn.Linear(input_dim, output_dim))
        else:
            layers.append(nn.Linear(input_dim, hidden_dim))
            for _ in range(num_layers - 2):
                layers.append(nn.Linear(hidden_dim, hidden_dim))
            layers.append(nn.Linear(hidden_dim, output_dim))
        self.layers = nn.ModuleList(layers)
        self.dropout_rate = float(dropout)
        self.drop = nn.Dropout(p=self.dropout_rate) if self.dropout_rate > 0.0 else nn.Identity()
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Apply ReLU + dropout between layers (no activation or dropout on final layer)
        for i, layer in enumerate(self.layers):
            x = layer(x)
            if i != len(self.layers) - 1:
                x = torch.relu(x)
                x = self.drop(x)
        return x

In [69]:
# Dataset loading and subsampling
ds = load_dataset(DATASET)
train_split = ds['train']
test_split = ds['test'] if 'test' in ds else ds['train']
train_n = min(TRAIN_SAMPLES, len(train_split))
test_n = min(TEST_SAMPLES, len(test_split))
def stratified_select(dataset, n, label_field='label', seed=123):
    labels = list(dataset[label_field])
    total = len(labels)
    if n >= total:
        return dataset.shuffle(seed=seed)
    classes = sorted(set(labels))
    counts = {c: labels.count(c) for c in classes}
    # Allocate per-class counts proportionally, with at least 1 when possible
    alloc = {}
    remaining = n
    for c in classes[:-1]:
        k = max(1, int(round(counts[c] / total * n)))
        k = min(k, counts[c])
        alloc[c] = k
        remaining -= k
    last = classes[-1]
    alloc[last] = min(counts[last], max(0, remaining))
    import random
    random.seed(seed)
    selected = []
    for c in classes:
        idxs = [i for i, lab in enumerate(labels) if lab == c]
        random.shuffle(idxs)
        take = alloc.get(c, 0)
        selected.extend(idxs[:take])
    # If short due to rounding, fill from remaining indices
    if len(selected) < n:
        remaining_idxs = [i for i in range(total) if i not in selected]
        random.shuffle(remaining_idxs)
        selected.extend(remaining_idxs[:(n - len(selected))])
    random.shuffle(selected)
    return dataset.select(selected)
# Apply stratified subsampling to train and test splits
train_split = stratified_select(train_split, train_n, seed=123)
test_split = stratified_select(test_split, test_n, seed=123)

In [70]:
display(train_split['text'][:5])

['This two-parter was excellent - the best since the series returned. Sure bits of the story were pinched from previous films, but what TV shows don\'t do that these days. What we got here was a cracking good sci-fi story. A great big (really scary) monster imprisoned at the base of a deep pit, some superb aliens in The Ood - the best "new" aliens the revived series has come up with, a set of basically sympathetic and believable human characters (complete with a couple of unnamed "expendable" security people in true Star Trek fashion), some large-scale philosophical themes (love, loyalty, faith, etc.), and some top-drawer special effects.<br /><br />I loved every minute of this.',
 "Stan Laurel and Oliver Hardy had extensive (separate) film careers before they were eventually teamed. For many of Ollie's pre-Stan films, he was billed on screen as Babe Hardy ... and throughout his adult life, Hardy was known to his friends as 'Babe'. While touring postwar Britain with Laurel in a music-h

In [71]:
# Tokenization and tensor conversion
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER, use_fast=True)
vocab_size = int(getattr(tokenizer, 'vocab_size', 30522))
def tok_batch(batch):
    # Use tokenizer's standard padding behavior (pad to longest in batch)
    enc = tokenizer(batch['text'], truncation=True, padding=True, max_length=MAX_LEN, return_attention_mask=False)
    return {'input_ids': enc['input_ids']}
train_tok = train_split.map(tok_batch, batched=True, remove_columns=[c for c in train_split.column_names if c != 'label'])
test_tok = test_split.map(tok_batch, batched=True, remove_columns=[c for c in test_split.column_names if c != 'label'])
train_ids = torch.tensor(train_tok['input_ids'], dtype=torch.int64)
train_y = torch.tensor(train_tok['label'], dtype=torch.int64)
test_ids = torch.tensor(test_tok['input_ids'], dtype=torch.int64)
test_y = torch.tensor(test_tok['label'], dtype=torch.int64)
train_x = train_ids.to(torch.float32) / float(vocab_size)
test_x = test_ids.to(torch.float32) / float(vocab_size)
num_classes = int(train_y.max().item() + 1)
input_dim = int(train_x.shape[1])

Map: 100%|██████████| 256/256 [00:00<00:00, 8119.77 examples/s]


In [72]:
# Training with validation and early stopping
model = LinearMLP(input_dim=input_dim, hidden_dim=HIDDEN_DIM, output_dim=num_classes, num_layers=NUM_LAYERS, dropout=DROPOUT_RATE).to(DEVICE)
# Create train/val split from train_x/train_y
n_train_total = train_x.shape[0]
val_n = int(n_train_total * VAL_RATIO)
if val_n > 0:
    # Stratified split to balance classes in train/val
    classes = torch.unique(train_y)
    train_idx_list = []
    val_idx_list = []
    for c in classes:
        idx_c = (train_y == c).nonzero(as_tuple=True)[0]
        idx_c = idx_c[torch.randperm(idx_c.size(0))]
        n_val_c = max(1, int(idx_c.size(0) * VAL_RATIO))
        val_idx_list.append(idx_c[:n_val_c])
        train_idx_list.append(idx_c[n_val_c:])
    val_idx = torch.cat(val_idx_list) if len(val_idx_list) > 0 else torch.tensor([], dtype=torch.long)
    train_idx = torch.cat(train_idx_list) if len(train_idx_list) > 0 else torch.tensor([], dtype=torch.long)
    # Shuffle final indices
    train_idx = train_idx[torch.randperm(train_idx.size(0))] if train_idx.numel() > 0 else train_idx
    val_idx = val_idx[torch.randperm(val_idx.size(0))] if val_idx.numel() > 0 else val_idx
    train_ds = TensorDataset(train_x[train_idx], train_y[train_idx])
    val_ds = TensorDataset(train_x[val_idx], train_y[val_idx])
else:
    train_ds = TensorDataset(train_x, train_y)
    val_ds = None
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False) if val_ds is not None else None
test_loader = DataLoader(TensorDataset(test_x, test_y), batch_size=BATCH_SIZE, shuffle=False)
optim = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()
best_val_acc = -1.0
best_state = None
epochs_no_improve = 0
model.train()
for epoch in range(1, EPOCHS+1):
    running = 0.0
    seen = 0
    for xb, yb in train_loader:
        xb = xb.to(DEVICE)
        yb = yb.to(DEVICE)
        optim.zero_grad(set_to_none=True)
        logits = model(xb)
        loss = loss_fn(logits, yb)
        loss.backward()
        optim.step()
        running += float(loss.item()) * int(yb.numel())
        seen += int(yb.numel())
    # Validation
    val_acc = None
    if val_loader is not None:
        model.eval()
        total_v = 0
        correct_v = 0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb = xb.to(DEVICE)
                yb = yb.to(DEVICE)
                logits = model(xb)
                pred = logits.argmax(dim=-1)
                correct_v += int((pred == yb).sum().item())
                total_v += int(yb.numel())
        val_acc = correct_v / max(total_v, 1)
    train_loss = running / max(seen, 1)
    # Check improvement on validation (or test if no val set)
    monitor_acc = val_acc if val_acc is not None else 0.0
    improved = (monitor_acc > best_val_acc)
    if improved:
        best_val_acc = monitor_acc
        # save CPU copy of state dict
        best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
    print(f'epoch={epoch} train_loss={train_loss:.4f} val_acc={val_acc if val_acc is not None else "N/A"}')
    model.train()
    if epochs_no_improve >= PATIENCE:
        print(f'Early stopping after {epoch} epochs (no improvement for {PATIENCE} epochs)')
        break
# restore best model state if available
if best_state is not None:
    model.load_state_dict(best_state)

epoch=1 train_loss=0.6948 val_acc=0.505
epoch=2 train_loss=0.6919 val_acc=0.505
epoch=3 train_loss=0.6898 val_acc=0.56
epoch=4 train_loss=0.6877 val_acc=0.52
epoch=5 train_loss=0.6819 val_acc=0.535
epoch=6 train_loss=0.6761 val_acc=0.555
epoch=7 train_loss=0.6731 val_acc=0.525
epoch=8 train_loss=0.6628 val_acc=0.515
Early stopping after 8 epochs (no improvement for 5 epochs)


In [73]:
# Final testing/evaluation
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for xb, yb in test_loader:
        xb = xb.to(DEVICE)
        yb = yb.to(DEVICE)
        logits = model(xb)
        pred = logits.argmax(dim=-1)
        correct += int((pred == yb).sum().item())
        total += int(yb.numel())
    print(f'final_test_acc={correct/max(total,1):.4f}')

final_test_acc=0.5273


In [75]:
# Expanded hyperparameter/configuration sweep (sampled from full grid)
import itertools, random, json, time
# parameter grids
lrs = [1e-2, 5e-3, 1e-3, 5e-4]
wds = [0.0, 1e-4, 1e-3]
dropouts = [0.0, 0.2, 0.3, 0.5]
num_layers_list = [1, 2, 3, 4]
hidden_dims = [64, 128, 256]
max_lens = [32, 64, 128]
padding_modes = [True, 'max_length']
# sampling configuration
FULL_GRID = list(itertools.product(lrs, wds, dropouts, num_layers_list, hidden_dims, max_lens, padding_modes))
random.seed(123)
random.shuffle(FULL_GRID)
N_CONFIGS = min(20, len(FULL_GRID))  # pick a sampled subset to keep runtime reasonable
SELECTED = FULL_GRID[:N_CONFIGS]
EPOCHS_SWEEP = 20
PATIENCE_SWEEP = 5
results = []
def prepare_tensors_for_config(max_len, padding_mode):
    # re-tokenize train/test splits for this config (keeps code simple and correct)
    def tok_batch_local(batch):
        if padding_mode == 'max_length':
            enc = tokenizer(batch['text'], truncation=True, padding='max_length', max_length=max_len, return_attention_mask=False)
        else:
            enc = tokenizer(batch['text'], truncation=True, padding=True, max_length=max_len, return_attention_mask=False)
        return {'input_ids': enc['input_ids']}
    tr_tok = train_split.map(tok_batch_local, batched=True, remove_columns=[c for c in train_split.column_names if c != 'label'])
    te_tok = test_split.map(tok_batch_local, batched=True, remove_columns=[c for c in test_split.column_names if c != 'label'])
    tr_ids = torch.tensor(tr_tok['input_ids'], dtype=torch.int64)
    te_ids = torch.tensor(te_tok['input_ids'], dtype=torch.int64)
    tr_y = torch.tensor(tr_tok['label'], dtype=torch.int64)
    te_y = torch.tensor(te_tok['label'], dtype=torch.int64)
    tr_x = tr_ids.to(torch.float32) / float(vocab_size)
    te_x = te_ids.to(torch.float32) / float(vocab_size)
    return tr_x, tr_y, te_x, te_y
start_time = time.time()
for (lr, wd, drop, nlayers, hdim, mlen, pad_mode) in SELECTED:
    torch.manual_seed(123)
    # prepare tensors for this config
    tr_x_cfg, tr_y_cfg, te_x_cfg, te_y_cfg = prepare_tensors_for_config(mlen, pad_mode)
    input_dim_cfg = int(tr_x_cfg.shape[1])
    # stratified train/val split (same logic as above)
    n_train_total = tr_x_cfg.shape[0]
    val_n = int(n_train_total * VAL_RATIO)
    if val_n > 0:
        classes = torch.unique(tr_y_cfg)
        train_idx_list = []
        val_idx_list = []
        for c in classes:
            idx_c = (tr_y_cfg == c).nonzero(as_tuple=True)[0]
            idx_c = idx_c[torch.randperm(idx_c.size(0))]
            n_val_c = max(1, int(idx_c.size(0) * VAL_RATIO))
            val_idx_list.append(idx_c[:n_val_c])
            train_idx_list.append(idx_c[n_val_c:])
        val_idx = torch.cat(val_idx_list) if len(val_idx_list) > 0 else torch.tensor([], dtype=torch.long)
        train_idx = torch.cat(train_idx_list) if len(train_idx_list) > 0 else torch.tensor([], dtype=torch.long)
        train_idx = train_idx[torch.randperm(train_idx.size(0))] if train_idx.numel() > 0 else train_idx
        val_idx = val_idx[torch.randperm(val_idx.size(0))] if val_idx.numel() > 0 else val_idx
        train_ds = TensorDataset(tr_x_cfg[train_idx], tr_y_cfg[train_idx])
        val_ds = TensorDataset(tr_x_cfg[val_idx], tr_y_cfg[val_idx])
    else:
        train_ds = TensorDataset(tr_x_cfg, tr_y_cfg)
        val_ds = None
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False) if val_ds is not None else None
    # build model for config
    model_s = LinearMLP(input_dim=input_dim_cfg, hidden_dim=hdim, output_dim=num_classes, num_layers=nlayers, dropout=drop).to(DEVICE)
    optim_s = torch.optim.Adam(model_s.parameters(), lr=lr, weight_decay=wd)
    loss_fn_s = nn.CrossEntropyLoss()
    best_val = -1.0
    best_state = None
    epochs_no_improve = 0
    for epoch in range(1, EPOCHS_SWEEP+1):
        model_s.train()
        running = 0.0
        seen = 0
        for xb, yb in train_loader:
            xb = xb.to(DEVICE)
            yb = yb.to(DEVICE)
            optim_s.zero_grad(set_to_none=True)
            logits = model_s(xb)
            loss = loss_fn_s(logits, yb)
            loss.backward()
            optim_s.step()
            running += float(loss.item()) * int(yb.numel())
            seen += int(yb.numel())
        # validation
        val_acc = None
        if val_loader is not None:
            model_s.eval()
            total_v = 0
            correct_v = 0
            with torch.no_grad():
                for xb, yb in val_loader:
                    xb = xb.to(DEVICE)
                    yb = yb.to(DEVICE)
                    pred = model_s(xb).argmax(dim=-1)
                    correct_v += int((pred == yb).sum().item())
                    total_v += int(yb.numel())
            val_acc = correct_v / max(total_v, 1)
        train_loss = running / max(seen, 1)
        monitor_acc = val_acc if val_acc is not None else 0.0
        if monitor_acc > best_val:
            best_val = monitor_acc
            best_state = {k: v.cpu().clone() for k, v in model_s.state_dict().items()}
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        if epochs_no_improve >= PATIENCE_SWEEP:
            break
    if best_state is not None:
        model_s.load_state_dict(best_state)
    # test evaluation
    model_s.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for xb, yb in DataLoader(TensorDataset(te_x_cfg, te_y_cfg), batch_size=BATCH_SIZE):
            xb = xb.to(DEVICE)
            yb = yb.to(DEVICE)
            pred = model_s(xb).argmax(dim=-1)
            correct += int((pred == yb).sum().item())
            total += int(yb.numel())
    test_acc = correct / max(total, 1)
    elapsed = time.time() - start_time
    cfg = dict(lr=lr, weight_decay=wd, dropout=drop, num_layers=nlayers, hidden_dim=hdim, max_len=mlen, padding=pad_mode)
    results.append({'config': cfg, 'best_val': best_val, 'test_acc': test_acc, 'elapsed_s': elapsed})
    print(f"config={cfg} best_val={best_val:.4f} test_acc={test_acc:.4f} elapsed={elapsed:.1f}s")
# summarize top results by test accuracy
results_sorted = sorted(results, key=lambda x: x['test_acc'], reverse=True)
print('\nTop results:')
for r in results_sorted[:5]:
    print(r)
# persist results
with open('sweep_results.json', 'w') as fh:
    json.dump(results_sorted, fh, indent=2)
print('Saved sweep_results.json')

Map: 100%|██████████| 256/256 [00:00<00:00, 8165.40 examples/s]


config={'lr': 0.001, 'weight_decay': 0.0, 'dropout': 0.3, 'num_layers': 3, 'hidden_dim': 64, 'max_len': 64, 'padding': True} best_val=0.5050 test_acc=0.4805 elapsed=0.7s
config={'lr': 0.0005, 'weight_decay': 0.001, 'dropout': 0.0, 'num_layers': 2, 'hidden_dim': 128, 'max_len': 64, 'padding': True} best_val=0.5500 test_acc=0.4727 elapsed=1.1s


Map: 100%|██████████| 256/256 [00:00<00:00, 8157.27 examples/s]


config={'lr': 0.0005, 'weight_decay': 0.001, 'dropout': 0.5, 'num_layers': 1, 'hidden_dim': 128, 'max_len': 128, 'padding': True} best_val=0.4800 test_acc=0.5312 elapsed=1.7s


Map: 100%|██████████| 256/256 [00:00<00:00, 7706.80 examples/s]


...

config={'lr': 0.01, 'weight_decay': 0.0001, 'dropout': 0.2, 'num_layers': 4, 'hidden_dim': 256, 'max_len': 32, 'padding': True} best_val=0.5000 test_acc=0.5000 elapsed=11.3s
config={'lr': 0.001, 'weight_decay': 0.001, 'dropout': 0.0, 'num_layers': 3, 'hidden_dim': 128, 'max_len': 32, 'padding': True} best_val=0.5200 test_acc=0.5000 elapsed=11.7s


Map: 100%|██████████| 2000/2000 [00:00<00:00, 9191.62 examples/s]


config={'lr': 0.001, 'weight_decay': 0.001, 'dropout': 0.3, 'num_layers': 3, 'hidden_dim': 256, 'max_len': 64, 'padding': 'max_length'} best_val=0.5100 test_acc=0.5273 elapsed=12.6s


Map: 100%|██████████| 2000/2000 [00:00<00:00, 8759.55 examples/s]


config={'lr': 0.01, 'weight_decay': 0.0001, 'dropout': 0.3, 'num_layers': 4, 'hidden_dim': 128, 'max_len': 128, 'padding': 'max_length'} best_val=0.5000 test_acc=0.5000 elapsed=13.3s

Top results:
{'config': {'lr': 0.0005, 'weight_decay': 0.0001, 'dropout': 0.5, 'num_layers': 1, 'hidden_dim': 128, 'max_len': 64, 'padding': True}, 'best_val': 0.515, 'test_acc': 0.54296875, 'elapsed_s': 9.984415054321289}
{'config': {'lr': 0.0005, 'weight_decay': 0.0001, 'dropout': 0.3, 'num_layers': 1, 'hidden_dim': 256, 'max_len': 64, 'padding': True}, 'best_val': 0.515, 'test_acc': 0.54296875, 'elapsed_s': 10.53002119064331}
{'config': {'lr': 0.01, 'weight_decay': 0.0001, 'dropout': 0.0, 'num_layers': 2, 'hidden_dim': 256, 'max_len': 64, 'padding': True}, 'best_val': 0.505, 'test_acc': 0.5390625, 'elapsed_s': 5.276341676712036}
{'config': {'lr': 0.01, 'weight_decay': 0.0, 'dropout': 0.2, 'num_layers': 3, 'hidden_dim': 128, 'max_len': 64, 'padding': True}, 'best_val': 0.51, 'test_acc': 0.53515625, 'ela